In [1]:
from connection import ConnectTask
import pandas as pd
import time
from pprint import pprint
c=ConnectTask()

In [18]:
with open("/data/zybank/BOC_20170705_1.txt",encoding='gbk') as f:
    data=f.read()
data.split('\\r\\n')

['商品订单号|支付流水号|银行商品订单号|银行支付流水号|订单金额|付款金额|交易类型|交易状态|第三方交易状态|订单支付时间|对账结果',
 '100000408812|10000040881200|100000000000109|DBS17070410015240400|1.15|1.15|支付|失败|不存在|2017-07-04 15:55:54|异常',
 '100000408830|10000040883000|100000000000117|DBS17070410015261900|48|48|支付|失败|不存在|2017-07-04 16:32:57|异常',
 '100000408847|10000040884700|100000000000121|DBS17070410015274300|1.15|1.15|支付|失败|不存在|2017-07-04 18:23:40|异常',
 '100000408852|10000040885200|100000000000122|DBS17070410015276600|5|5|支付|成功|成功|2017-07-04 18:27:56|平账',
 '100000408840|10000040884000|100000000000120|DBS17070410015271900|1.15|1.15|支付|失败|不存在|2017-07-04 17:39:53|异常',
 '100000408860|10000040886000|100000000000124|DBS17070410015280100|1.15|1.15|支付|成功|成功|2017-07-04 19:15:59|平账',
 '100000408861|10000040886100|100000000000125|DBS17070410015280700|5|5|支付|成功|成功|2017-07-04 19:16:16|平账',
 '100000408826|10000040882600|100000000000113|DBS17070410015260500|1.15|1.15|支付|失败|不存在|2017-07-04 16:26:39|异常',
 '100000408828|10000040882800|100000000000115|DBS17

### 统计平安一账通19-21日取消订单里面，每个用户id取消次数

In [94]:
start=time.mktime((2017,6,1,0,0,0,0,0,0))*1000
end=time.mktime((2017,6,1,12,0,0,0,0,0))*1000

In [95]:
data=c.pro.order_lite_list.find({"appCode":"pingan-one",
                                "createdAt":{"$gte":start,"$lte":end},
                                "status":8})

In [96]:
count={}
for item in data:
    count.setdefault(item['userId'],0)
    count[item['userId']]+=1
count_list=[(k,v) for k,v in count.items()]
pprint(count_list)

[('509601431085239728', 1),
 ('958761476673926064', 1),
 ('810401304855424883', 1),
 ('002161496246791655', 1),
 ('999999052821013362', 3),
 ('520491465437083428', 2),
 ('037681365778863480', 1),
 ('382091478137993182', 1),
 ('631811479976854019', 3),
 ('321761486461680549', 1),
 ('028581489677844697', 1),
 ('191880000000228792', 1),
 ('637331304094831447', 2),
 ('421441471316829990', 6),
 ('365701304127392367', 2),
 ('491831372147493849', 1),
 ('474601496260034134', 1),
 ('362571313159818749', 3),
 ('017021411557191833', 2),
 ('687141303870040980', 1),
 ('727851473650623191', 1),
 ('752421307260616523', 2),
 ('727441464367361869', 1),
 ('704171489328952628', 1),
 ('002701468241087244', 1),
 ('037841378266868348', 2),
 ('902171306460731350', 1),
 ('599151364953161902', 7),
 ('165771481694429963', 1),
 ('488421322098244061', 1),
 ('991511202001123327', 8),
 ('997131418733522219', 2),
 ('203101304511978962', 1),
 ('321531356966643727', 1),
 ('307981463579846551', 2),
 ('17371130615818315

In [97]:
import tablib
data_tab=tablib.Dataset(headers=["用户ID","取消次数"])
for toup in count_list:
    data_tab.append(toup)

In [98]:
with open("6.1平安一账通统计用户下单取消次数.xls","wb") as f:
    f.write(data_tab.xls)

In [73]:
user_ids=c.pro.order_lite_list.distinct("userId",{"appCode":"pingan-one"})

In [74]:
len(user_ids)

670

In [75]:
len(count_list)

486

In [86]:
now_time=time.mktime((2017,5,19,1,10,0,0,0,0))
mdb_data = c.pro.order_lite_list.aggregate([
        {'$match': {'appCode': "allinpay"}},
        {'$match': {'$or': [{'createdAt': {'$gte': (now_time - 60 * 60) * 1000, '$lt': now_time * 1000}},
                            {'completedAt': {'$gte': (now_time - 60 * 60) * 1000, '$lt': now_time * 1000}}
                            ]
                    }
         }
    ])

In [83]:
mdb_data = c.pro.order_lite_list.find(
        {'appCode': "pingan-one",
        'completedAt': {'$gte': (now_time - 60 * 60) * 1000, '$lt': now_time * 1000}}
)

In [87]:
for item in mdb_data:
    pprint(item)

{'_id': ObjectId('591da0cbad463931de53e84d'),
 'appCode': 'allinpay',
 'channel': 'ele',
 'completedAt': 1495124758218.7058,
 'createdAt': 1495113931678.0,
 'customerId': '260',
 'customerOrderId': '201705182125370452',
 'customerUserId': '201512151448601',
 'displayStatus': '已完成',
 'id': '1000000088848',
 'name': '一点点（东百元洪店）',
 'orderType': 'waimai',
 'payChannel': 'allinpay_qianbao',
 'price': 41,
 'status': 4,
 'updatedAt': 1495113931678.0,
 'userId': '201512151448601',
 'userPhone': '15060038518'}


In [85]:
(now_time - 60 * 60) * 1000<1495124758218<now_time * 1000

True

In [17]:
c.pro.order_huafei.find_one({"appCode":"pingan-one","status":4,"checkData":{"$exists":True}})

In [30]:
c.pro.order_lite_list.find_one({"appCode":"pingan-one","status":4,'id': '1000000404845'})

In [106]:
time.strftime("%Y%m%d", time.localtime(time.time() - 3600*15))

'20170604'

In [112]:
",".join(['1','2'])

'1,2'

In [138]:
check_date = time.strftime("%Y%m%d", time.localtime(time.time() - 3600*20))
mongo_data=[]
for order_type in c.pro["order_lite_list"].distinct("orderType"):
    if not order_type:
        # order_type maybe is None
        continue

    for item in c.pro["order_{}".format(order_type)].find({"appCode": "pingan-one", "status": 4},
                                                              {"_id": 0, "id": 1, "paidList": 1, "price": 1}).limit(2):
        item["paidList"]=",".join(item["paidList"])
        mongo_data.append(item)

    account_data = pd.DataFrame(mongo_data)

In [132]:
item

{'id': '1000000086854', 'paidList': ['100000008685400'], 'price': 49.8}

In [140]:
account_data["price"].sum()

958.75

In [141]:
account_data["id"].count()

10

In [171]:
check_date = time.strftime("%Y%m%d", time.localtime(time.time() - 3600*24))
file_name = "otosaas_pingan_{}.txt".format(check_date)
account_data = {"priceSum": 0, "countSum": 0}
mongo_data = []
with open(file_name, "w+") as f:
    f.write(check_date+"\n")
    pass
for order_type in c.pro["order_lite_list"].distinct("orderType"):
    if not order_type:
        # order_type maybe is None
        continue
    database = "order_{}".format(order_type)
    for item in c.pro[database].find({"appCode": "pingan-one", "status": 4},
                                        {"_id": 0, "id": 1, "paidList": 1, "price": 1}):
        item["paidList"] = ",".join(item.get("paidList", " "))
        with open(file_name, 'a+') as f:
            f.write("\n" + " | ".join([item["id"], item["paidList"], str(item["price"]), '4']))
        account_data["priceSum"] += item["price"]
        account_data["countSum"] += 1

with open(file_name, "r+") as f:
    f.write("|".join([check_date, str(account_data["countSum"]), "{:.2f}".format(account_data["priceSum"])])+"\n")

In [170]:
with open(file_name, "r+") as f:
    f.write("|".join([check_date, str(account_data["countSum"]), "{:.2f}".format(account_data["priceSum"])])+"\n")

In [190]:
check_date = time.strftime("%Y%m%d", time.localtime(time.time() - 3600*24))
file_name = "otosaas_pingan_{}.txt".format(check_date)
account_data = {"priceSum": 0, "countSum": 0}
mongo_data = []
with open(file_name, "w+") as f:
    pass
for order_type in c.pro["order_lite_list"].distinct("orderType"):
    if not order_type:
        # order_type maybe is None
        continue
    database = "order_{}".format(order_type)
    for item in c.pro[database].find({"appCode": "pingan-one", "status": 4},
                                        {"_id": 0, "id": 1, "paidList": 1, "price": 1}):
        item["paidList"] = ",".join(item.get("paidList", " "))
        with open(file_name, 'a+') as f:
            f.write(" | ".join([item["id"], item["paidList"], "{:.2f}".format(item["price"]), '4']) + "\n")
        account_data["priceSum"] += item["price"]
        account_data["countSum"] += 1
lines = []
with open(file_name) as f:
    for line in f:
        lines.append(line)
lines.insert(0, "|".join([check_date, str(account_data["countSum"]), "{:.2f}".format(account_data["priceSum"])])+"\n")
with open(file_name, "w+") as f:
    f.write("".join(lines))


## 平安加入checkdata字段

In [3]:
import time 
for item in c.pro.order_huafei.find({"appCode":"pingan-one","status":4}):
    c.pro.order_huafei.update_one({"id":item["id"]},{"$unset":{"checkDate":''}})
    print(item["id"])

1000000086854
1000000086855
1000000086872
1000000086917
1000000086918
1000000086920
1000000086933
1000000086937
1000000086966
1000000086989
1000000087148
1000000087961
1000000087994
1000000088125
1000000088404
1000000088619
1000000088622
1000000088777
1000000088783
1000000088906
1000000088917
1000000088923
1000000088927
1000000088928
1000000088929
1000000088933
1000000088939
1000000088940
1000000088947
1000000088963
1000000088968
1000000088969
1000000088980
1000000089077
1000000089095
1000000089097
1000000089160
1000000089177
1000000089204
1000000089213
1000000089256
1000000089264
1000000089297
1000000089328
1000000089334
1000000089348
1000000089357
1000000089399
1000000089401
1000000089408
1000000089410
1000000089418
1000000089420
1000000089498
1000000089518
1000000089520
1000000089612
1000000089635
1000000089636
1000000089658
1000000089708
1000000089729
1000000089735
1000000089758
1000000089777
1000000089813
1000000089815
1000000089971
1000000089992
1000000090005
1000000090064
100000